<a href="https://colab.research.google.com/github/nkashanov/DS/blob/main/z-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Тест А/Б метрик

In [2]:
import pandas as pd
import numpy as np

In [3]:
np.random.binomial(1, 0.3, size = 10)

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [4]:
a = np.random.binomial(1, 0.03, size = 1000).mean()
b = np.random.binomial(1, 0.05, size = 1000).mean()

In [5]:
a, b

(0.022, 0.06)

Может получиться ли что в первый группе конверсия станет больше чем во второй?

In [6]:
n = 1000
result = []
for i in range(n):
  a = np.random.binomial(1, 0.03, size = 1000).mean()
  b = np.random.binomial(1, 0.05, size = 1000).mean()
  result.append((a,b))

In [7]:
df = pd.DataFrame(result, columns = ['a', 'b'])

In [9]:
df[df['a'] > df['b']]

,a,b
152,0.043,0.032
374,0.034,0.028
481,0.045,0.041
623,0.039,0.036
783,0.042,0.038
822,0.036,0.033
869,0.028,0.027
923,0.048,0.047
956,0.041,0.037


Да такое может происходить - нужно проводить **стат. тест**
Нужны метрики 3 основные **TPR, FPR, MDE**


С помощью **статистики** эти метрики можно посчитать заранее

и проверить с помощью метода **Монте-Карло - это выше**

In [10]:
  a = np.random.binomial(1, 0.03, size = 1484).mean() #массив со кол-во значений 1484, 1 - ответ бинарен 0/1
  b = np.random.binomial(1, 0.05, size = 1484).mean()

In [11]:
a * 1484, b * 1484

(51.0, 80.0)

 np.random.binomial(1, p, size) даёт массив 0 и 1

 np.random.binomial(n, p) даёт одно число — сумму успехов.

In [12]:
np.random.binomial(1484, 0.03) #заходит 1484 человека, вероятность покупки 3%
#получим кол-во людей которые купили

54

In [13]:
  a = np.random.binomial(1484, 0.03)
  b = np.random.binomial(1484, 0.05)

In [14]:
a, b

(41, 86)

In [15]:
from statsmodels.stats.proportion import proportions_ztest

In [16]:
def test(conv_a, conv_b, size_a, size_b, significance=0.05): #TPR (мощность) заложена в размере выборки, мощность закладываем на этапе расчета
    _, p_value = proportions_ztest([conv_a, conv_b],         #если уменьшим размер выборки мощность упадет
                                   [size_a, size_b],
                                   alternative='two-sided')
    return p_value < significance


In [17]:
test(44, 78, 1484, 1484)

True

## TPR(sensetivety) тестим

In [19]:
n = 10000
result = []
for _ in range(n):
  a = np.random.binomial(1484, 0.03)
  b = np.random.binomial(1484, 0.05)
  result.append((a,b))

In [20]:
df = pd.DataFrame(result, columns = ['a', 'b'])

In [22]:
df['test'] = df.apply(lambda row: test(row['a'], row['b'], 1484, 1484), axis = 1)

In [23]:
df

,a,b,test
0,44,55,False
1,54,68,False
2,40,73,True
3,39,77,True
4,40,77,True
...,...,...,...
9995,43,77,True
9996,42,81,True
9997,56,72,False
9998,36,85,True


In [24]:
df['test'].mean() #должно получится около 80%

0.8004

Нужно понимать, что метод **Монте-Карло** так же имеет свою точность

Точность заложена в **кол-ве виртуальных** экспериментов

Обычно достаточно 1000 или 10 000 экспериментов

## FPR тестим

In [25]:
from tqdm.notebook import tqdm

In [26]:
n = 10000
result = []
for _ in tqdm(range(n)):
  a = np.random.binomial(1484, 0.03)
  b = np.random.binomial(1484, 0.03)
  result.append((a,b))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [27]:
df = pd.DataFrame(result, columns = ['a', 'b'])

In [29]:
#добавим прогресс-бар для Пандас

In [30]:
from tqdm import tqdm

In [31]:
tqdm.pandas()

In [32]:
df['test'] = df.progress_apply(lambda row: test(row['a'], row['b'], 1484, 1484), axis = 1)

100%|██████████| 10000/10000 [00:06<00:00, 1576.21it/s]


In [33]:
df['test'].mean() #должно получится около 5%

0.0483

## MDE протестим (минимальный эффект который обнаруживает тест)

чтобы его проверить нужно просто перебрать

разницу между двумя выборками

**разный шаг, разный TPR, а нам надо желаемый** **TPR**

In [34]:
from tqdm.notebook import tqdm

In [35]:
n = 10000
result = []
for _ in tqdm(range(n)):
  a = np.random.binomial(1484, 0.03)
  b = np.random.binomial(1484, 0.05)
  result.append((a,b))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [36]:
df = pd.DataFrame(result, columns = ['a', 'b'])

In [37]:
from tqdm import tqdm

In [38]:
tqdm.pandas()

In [39]:
df['test'] = df.progress_apply(lambda row: test(row['a'], row['b'], 1484, 1484), axis = 1)

100%|██████████| 10000/10000 [00:02<00:00, 3795.93it/s]


In [40]:
df['test'].mean() #должно получится около 5%

0.8008

## Тест сайта [evanmiller](//https://www.evanmiller.org/ab-testing/sample-size.html)

In [150]:
def test(conv_a, conv_b, size_a, size_b, significance=0.05): #TPR (мощность) заложена в размере выборки, мощность закладываем на этапе расчета
    _, p_value = proportions_ztest([conv_a, conv_b],         #если уменьшим размер выборки мощность упадет
                                   [size_a, size_b],
                                   alternative='two-sided')
    return p_value < significance


### **TPR**

In [158]:
from tqdm.notebook import tqdm

In [159]:
n = 10000
result = []
for _ in tqdm(range(n)):
  a = np.random.binomial(3792, 0.3)
  b = np.random.binomial(3792, 0.32)
  result.append((a,b))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [160]:
df = pd.DataFrame(result, columns = ['a', 'b'])

In [161]:
from tqdm import tqdm

In [162]:
tqdm.pandas()

In [163]:
df['test'] = df.progress_apply(lambda row: test(row['a'], row['b'], 3792, 3792, significance=0.1), axis = 1)

100%|██████████| 10000/10000 [00:01<00:00, 6128.80it/s]


In [164]:
df['test'].mean()

0.5951

### FPR

In [165]:
n = 10000
result = []
for _ in range(n):
  a = np.random.binomial(3792, 0.3)
  b = np.random.binomial(3792, 0.3)
  result.append((a,b))

In [166]:
df = pd.DataFrame(result, columns = ['a', 'b'])

In [169]:
df['test'] = df.apply(lambda row: test(row['a'], row['b'], 3792, 3792, significance=0.1), axis = 1)

In [170]:
df['test'].mean()

0.0992

**Не забываем менять значимость FPR**

#ДЛЯ ПРАКТИКА

In [173]:
import numpy as np
from statsmodels.stats.power import NormalIndPower
import statsmodels.api as sm

# Определяем параметры A/B-теста
baseline = 0.03
mde = 0.02
ratio = 1
power = 0.8
alpha = 0.05

# Создаем объект для анализа мощности теста
analysis = NormalIndPower()

effect_size = sm.stats.proportion_effectsize(baseline, baseline + mde)

result = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=ratio, alternative= 'two-sided')

sample_size = int(np.ceil(result))


sample_size, sample_size * ratio


(1484, 1484)

Далее проверяем с помощью Монте-Карло